In [90]:
import spacy
from spacy.pipeline import DependencyParser
from spacy.symbols import nsubj, VERB

In [91]:
nlp=spacy.load('en')

In [115]:
doc=nlp(u"I want to go to the zoo with my friends")

In [81]:
for np in doc.noun_chunks:
    print(np.text)
    print(np.root.dep_)

it
nsubj


In [82]:
DependencyParser(doc.vocab)

In [106]:
verbs = set()
for possible_subject in doc:
    if possible_subject.dep == nsubj : #and possible_subject.head.pos == VERB:
        verbs.add((possible_subject.head,possible_subject,possible_subject.dep_))

In [107]:
verbs

{(go, it, 'nsubj')}

In [116]:
word={}
for words in doc:
    if words.dep_ !='ROOT':
        word[words]=(words.head,words.dep_)
        print(words.head,words,words.dep_)

want I nsubj
go to aux
want go xcomp
go to prep
zoo the det
to zoo pobj
go with prep
friends my poss
with friends pobj


In [110]:
import os
import pandas as pd
os.chdir('/Users/builduser/Documents/Enconversion')

In [148]:
import glob
import numpy as np
file_list=(glob.glob("/Users/builduser/Documents/Enconversion/en/*.txt"))

In [130]:
unls=[]
words=[]
for files in file_list:
    file=open(files).readlines()
    for i,x in enumerate(file):
        if i%2!=0:
            unls.append(x.strip('\n'))
        else:
            words.append(x.strip(' -->> .-\n'))

    

In [145]:
words=pd.DataFrame(words,columns=['words'])
unls=pd.DataFrame(unls,columns=['unls'])
data=pd.concat([words,unls],axis=1)

In [218]:
data.head()

,words,unls,agt
0,II- I see,"agt(2002129289,00.@1)",1
1,I agreed,"agt(2000805376.@past,00.@1)",1
2,I ate it,"agt(2001168468.@past,00.@1);obj(2001168468.@pa...",1
3,I can go,"agt(2101835496.@ability,00.@1)",1
4,I did it,"agt(2002560585.@past,00.@1);obj(2002560585.@pa...",1


In [231]:
data['Attr_Count']=data.unls.str.count("\(")

In [232]:
data

,words,unls,agt,Attr_Count
0,II- I see,"agt(2002129289,00.@1)",1,1.0
1,I agreed,"agt(2000805376.@past,00.@1)",1,1.0
2,I ate it,"agt(2001168468.@past,00.@1);obj(2001168468.@pa...",1,2.0
3,I can go,"agt(2101835496.@ability,00.@1)",1,1.0
4,I did it,"agt(2002560585.@past,00.@1);obj(2002560585.@pa...",1,2.0
5,I got it,"agt(2002210855.@past,00.@1);obj(2002210855.@pa...",1,2.0
6,I will try,"agt(2002530167.@future,00.@1)",1,1.0
7,May I go?,"agt(2101835496.@interrogative.@hypothesis,00.@1)",1,1.0
8,I am sick,"agt(2002604760,00.@1);obj(2002604760,3002541302)",1,2.0
9,I am full,"agt(2002604760,00.@1);obj(2002604760,3001083157)",1,2.0


In [233]:
data['agt']=np.where(data.unls.str.contains('agt'),1,0)
data['obj']=np.where(data.unls.str.contains('obj'),1,0)

In [266]:
from sklearn.feature_extraction.text import CountVectorizer

In [278]:
count=CountVectorizer()
count.fit(data.words.values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [279]:
array=count.transform(data.words)

In [280]:
train=array.toarray()

In [298]:
y=data.agt.values

In [299]:
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix
#from sklearn.

In [300]:
x_train,x_test,y_train,y_test=train_test_split(train,y)

In [301]:
model=MultinomialNB()

In [302]:
model.fit(x_train,y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [303]:
pred=model.predict(x_test)

In [304]:
accuracy_score(pred,y_test)

0.88788128606760097

In [305]:
confusion_matrix(y_test,pred)

array([[134, 125],
       [ 11, 943]])